In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.7 MB/s eta 0:00:00


In [2]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false

env: XLA_PYTHON_CLIENT_PREALLOCATE=false


In [3]:
%env XLA_PYTHON_CLIENT_ALLOCATOR=platform

env: XLA_PYTHON_CLIENT_ALLOCATOR=platform


In [4]:
from transformers import FlaxAutoModelForCausalLM, AutoTokenizer
import jax

In [5]:
from transformers import FlaxGPTJForCausalLM #?

In [5]:
model_name = "EleutherAI/gpt-j-6B"
#model_name = "EleutherAI/gpt-neo-125M"
#model_name = "EleutherAI/gpt-neo-1.3B"
#model_name = "EleutherAI/gpt-neo-2.7B"

In [6]:
model = FlaxAutoModelForCausalLM.from_pretrained(
    model_name,
    revision="float16",
    dtype=jax.numpy.float16,
    from_pt=True
)

Some weights of the model checkpoint at EleutherAI/gpt-j-6B were not used when initializing FlaxGPTJForCausalLM: {('transformer', 'h', '9', 'attn', 'bias'), ('transformer', 'h', '4', 'attn', 'masked_bias'), ('transformer', 'h', '6', 'attn', 'masked_bias'), ('transformer', 'h', '5', 'attn', 'bias'), ('transformer', 'h', '26', 'attn', 'bias'), ('transformer', 'h', '21', 'attn', 'bias'), ('transformer', 'h', '27', 'attn', 'masked_bias'), ('transformer', 'h', '13', 'attn', 'bias'), ('transformer', 'h', '9', 'attn', 'masked_bias'), ('transformer', 'h', '5', 'attn', 'masked_bias'), ('transformer', 'h', '20', 'attn', 'masked_bias'), ('transformer', 'h', '26', 'attn', 'masked_bias'), ('transformer', 'h', '21', 'attn', 'masked_bias'), ('transformer', 'h', '16', 'attn', 'bias'), ('transformer', 'h', '13', 'attn', 'masked_bias'), ('transformer', 'h', '8', 'attn', 'bias'), ('transformer', 'h', '14', 'attn', 'bias'), ('transformer', 'h', '19', 'attn', 'bias'), ('transformer', 'h', '3', 'attn', 'bia

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
!nvidia-smi --query-gpu=utilization.memory,memory.total,memory.free,memory.used --format=csv

utilization.memory [%], memory.total [MiB], memory.free [MiB], memory.used [MiB]
0 %, 40960 MiB, 28052 MiB, 12461 MiB


In [9]:
!nvidia-smi

Sat Sep  2 09:28:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    53W / 350W |  12461MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
# https://huggingface.co/docs/transformers/main/en/main_classes/model#transformers.FlaxPreTrainedModel.to_fp16
# or bf16
model.params = model.to_fp16(model.params)

In [13]:
!nvidia-smi --query-gpu=utilization.memory,memory.total,memory.free,memory.used --format=csv

utilization.memory [%], memory.total [MiB], memory.free [MiB], memory.used [MiB]
0 %, 40960 MiB, 28052 MiB, 12461 MiB


In [14]:
model

In [15]:
tokenizer

GPT2TokenizerFast(name_or_path='EleutherAI/gpt-j-6B', vocab_size=50257, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

In [34]:
prompt = """Generate SQL query from the text inside triple backticks
```select all the users from table 'users' older than 20 years```
"""

In [35]:
inputs = tokenizer(prompt, return_tensors="jax")

In [36]:
inputs

{'input_ids': Array([[ 8645,   378, 16363, 12405,   422,   262,  2420,  2641, 15055,
          736,    83,  3378,   198, 15506,    63, 19738,   477,   262,
         2985,   422,  3084,   705, 18417,     6,  4697,   621,  1160,
          812, 15506,    63,   198]], dtype=int32), 'attention_mask': Array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)}

https://huggingface.co/blog/how-to-generate

https://huggingface.co/docs/transformers/main_classes/text_generation#generation

https://huggingface.co/docs/transformers/generation_strategies



In [39]:
generated_ids = model.generate(
    **inputs,
    do_sample=True,
    num_beams=1,
    max_new_tokens=300,
    temperature=0.7,
    pad_token_id = model.config.eos_token_id,
    prng_key=jax.random.PRNGKey(4232),
    no_repeat_ngram_size=2)

In [40]:
generated_text = tokenizer.decode(generated_ids['sequences'].squeeze(0))
print(generated_text)

Generate SQL query from the text inside triple backticks
```select all the users from table 'users' older than 20 years```

```
SELECT * FROM `users` WHERE age > 20
```

```
SELECT * FROM `users` WHERE age > `20 years`
```

```
SELECT * FROM `users` WHERE `age` > 20`
```

<!--end-query-->

## Generate SQL query from the text inside double backticks
```select all the users from table `users` older than 20 years```

```
SELECT * FROM `users` WHERE `age` > 20
```

```
SELECT * FROM `users` WHERE `age` > `20 years`
```

```
SELECT * FROM `users` WHERE `age` > 20`
```

<!--end-query-->

## Generate SQL query from the text inside single backticks
```select all the users from table `users` older than 20 years```

```
SELECT * FROM `users` WHERE `age` > 20
```

```
SELECT * FROM `users` WHERE age > `20 years`
```

```
SELECT * FROM `users` WHERE `age` > 20`
```

<!--end-query-->

## Generate SQL query from the text inside single backticks with escaped quotes
```select all the users
